# Event-Driven Architecture: MinIO Integrations - Automating Bucket Events with Webhooks and PostgreSQL

### Proof of Concept Summary for Blog Review

This article presents a comprehensive Proof of Concept (PoC) that demonstrates the integration of Flask, a micro web framework, with MinIO, an object storage server, and PostgreSQL, a relational database, for efficient event data handling and storage. The primary goal of this PoC is to illustrate how these three technologies can be seamlessly combined to build a robust, scalable system capable of managing event data in real-time.

Key components of this integration include:
- **Pydantic Classes**: Leveraged for data validation and settings management, ensuring robust data handling.
- **MinIO Client Initialization**: Demonstrates the use of MinIO for object storage, essential for handling large volumes of data.
- **PostgreSQL Database Integration**: Showcases how to reliably store and manage structured event data.
- **Flask Web Server**: Serves as the backbone of the application, handling HTTP requests and routing them to the appropriate processing functions.

The PoC also explores:
- **Event Processing and Serialization**: Critical for ensuring that data is correctly formatted and stored.
- **Error Handling Mechanisms**: Demonstrating robustness and reliability of the system in adverse conditions.
- **Practical Application**: Through the simulation of an event, we provide a real-world scenario that showcases the system’s capabilities.

The expected outcome of this PoC is to validate the feasibility and effectiveness of integrating Flask, MinIO, and PostgreSQL for event data management. It is a compelling demonstration for developers and architects looking to build scalable and efficient applications using these technologies.

As we prepare this PoC for review, we aim to provide a detailed, practical example that not only showcases technical integration but also offers insights into its real-world applicability and scalability. This article will be an invaluable resource for professionals seeking to implement similar solutions in their projects or enterprises.

---

# Setup the MinIO and Postgres Services with Docker Compose

## Write docker-compose.yaml to local directory

In [ ]:
%%writefile docker-compose.yaml
version: '3.8'
services:
  Flaskapp: # This is where we can assign a hostname
    build: .
    ports:
      - "5000:5000"
    depends_on:
      - minio
      - postgres
      - redis

  minio:
    image: minio/minio
    environment:
      MINIO_ACCESS_KEY: minio
      MINIO_SECRET_KEY: minio123
    command: server /data
    ports:
      - "9000:9000"
    volumes:
      - minio_data:/data

  postgres:
    image: postgres:alpine
    environment:
      POSTGRES_DB: postgres
      POSTGRES_USER: myuser
      POSTGRES_PASSWORD: mypassword
    ports:
      - "5432:5432"
    volumes:
      - postgres_data:/var/lib/postgresql/data

volumes:
  minio_data:
  postgres_data:

# If MinIO is already running

You can deploy a Postgres Container like so:

In [ ]:
!docker run -d \
    --name postgres \
    -e POSTGRES_DB=postgres \
    -e POSTGRES_USER=myuser \
    -e POSTGRES_PASSWORD=mypassword \
    -p 5432:5432 \
    -v postgres_data:/var/lib/postgresql/data \
    postgres:alpine

## Check if containers are running

In [10]:
!docker ps -f name=minio
!docker ps -f name=postgres

CONTAINER ID   IMAGE                 COMMAND                  CREATED        STATUS       PORTS                                            NAMES
0e113865e67e   quay.io/minio/minio   "/usr/bin/docker-entâ€¦"   4 months ago   Up 4 hours   0.0.0.0:9000->9000/tcp, 0.0.0.0:9090->9090/tcp   minio
CONTAINER ID   IMAGE             COMMAND                  CREATED      STATUS      PORTS                    NAMES
c580a2f2a48c   postgres:alpine   "docker-entrypoint.sâ€¦"   6 days ago   Up 6 days   0.0.0.0:5432->5432/tcp   postgres


## Create the table in the container

In [1]:
!docker exec postgres psql -U myuser -d postgres -c "CREATE TABLE IF NOT EXISTS events (id SERIAL PRIMARY KEY, event_type VARCHAR(255), data JSON);"

Error response from daemon: container c580a2f2a48ce7612d90bee14fd827fd2c994605119d824cf1ce69178376d99d is not running


# The API Development Code for Integrating Numerous Clients & Services with MinIO

Ive added lots of comments to outline each step for whoever is reading this:

In [7]:
from pydantic import BaseModel
from flask import Flask, request
from minio import Minio
import psycopg2
import json

# Pydantic configuration class for MinIO client
class MinioClientConfig(BaseModel):
    endpoint: str = '192.168.0.25:9000'  # Using container hostname
    access_key: str = 'minio'
    secret_key: str = 'minio123'

# Pydantic configuration class for PostgreSQL client
class PostgresClientConfig(BaseModel):
    host: str = 'localhost'  # Using container hostname
    port: int = 5432
    user: str = 'myuser'
    password: str = 'mypassword'
    database: str = 'postgres'

# Pydantic class for event structure
class Event(BaseModel):
    event_type: str
    data: dict

# Create an instance of MinioClientConfig
minio_config = MinioClientConfig()

# Initialize MinIO client using the instance
minio_client = Minio(minio_config.endpoint, 
                     access_key=minio_config.access_key, 
                     secret_key=minio_config.secret_key)

# Create an instance of PostgresClientConfig
postgres_config = PostgresClientConfig()

# Initialize PostgreSQL connection using the instance
pg_conn = psycopg2.connect(
    host=postgres_config.host,
    port=postgres_config.port,
    user=postgres_config.user,
    password=postgres_config.password,
    dbname=postgres_config.database)

## Uncomment the following for a function that create_events_table which isnt necessary if the notebook is ran in order

# def create_events_table():
#     create_table_query = """
#     CREATE TABLE IF NOT EXISTS events (
#         id SERIAL PRIMARY KEY,
#         event_type VARCHAR(255),
#         data JSON
#     );
#     """
#     try:
#         with pg_conn.cursor() as cur:
#             cur.execute(create_table_query)
#             pg_conn.commit()
#             print("Table 'events' has been successfully.")
#     except Exception as e:
#         print(f"Error creating table: {e}")

# Flask app initialization
app = Flask(__name__)

# Route for handling events
@app.route('/event', methods=['POST'])
def handle_event():
    event_data = request.json
    event = Event(**event_data)
    process_event(event)
    return "Event processed"

# Function to process events
def process_event(event: Event):
    # Serialize data to JSON
    json_data = json.dumps(event.data)

    # Process event and log data in PostgreSQL
    try:
        with pg_conn.cursor() as cur:
            cur.execute("INSERT INTO events (event_type, data) VALUES (%s, %s)", 
                        (event.event_type, json_data))
            pg_conn.commit()
            print("Event Notification for 'simulated_event_data' has been inserted successfully into 'Events' Table.")
    except Exception as e:
        pg_conn.rollback()  # Roll back the transaction in case of an error
        print(f"Error processing event: {e}")
        
# Simulate an event for demonstration
simulated_event_data = {
    'event_type': 'file_uploaded',
    'data': {
        'file_name': 'example.txt',
        'file_size': '1024',
        'bucket_name': 'test-bucket'
    }
}

# Create Event instance with simulated data
simulated_event = Event(**simulated_event_data)

# Process the simulated event
process_event(simulated_event)

# Run the Flask app if this script is the main program
if __name__ == "__main__":
    #create_events_table()
    app.run(debug=True, port=5000)

Event Notification for 'simulated_event_data' has been inserted successfully into 'Events' Table.
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\david\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Viewing the Results

In [8]:
!docker exec postgres psql -U myuser -d postgres -c "SELECT * FROM events;"

 id |  event_type   |                                      data                                       | event_time | key | value 
----+---------------+---------------------------------------------------------------------------------+------------+-----+-------
  1 | file_uploaded | {"file_name": "example.txt", "file_size": "1024", "bucket_name": "test-bucket"} |            |     | 
  2 | file_uploaded | {"file_name": "example.txt", "file_size": "1024", "bucket_name": "test-bucket"} |            |     | 
  3 | file_uploaded | {"file_name": "example.txt", "file_size": "1024", "bucket_name": "test-bucket"} |            |     | 
  4 | file_uploaded | {"file_name": "example.txt", "file_size": "1024", "bucket_name": "test-bucket"} |            |     | 
  5 | file_uploaded | {"file_name": "example.txt", "file_size": "1024", "bucket_name": "test-bucket"} |            |     | 
  6 | file_uploaded | {"file_name": "example.txt", "file_size": "1024", "bucket_name": "test-bucket"} |            |    